In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np

In [6]:
titanic = pd.read_csv("titanic.csv")
targets = titanic.Survived
data = titanic.drop(columns='Survived')

X_train, X_test, y_train, y_test = train_test_split(data,
                                                    targets,
                                                    test_size=0.8, 
                                                    random_state=0)

Так как нам нужно и обучить базовые алгоритмы на тренировочном сете и сделать на этих же данных предсказания для обучения мета-алгоритма, разделим тренировочнфе данные еще на 2 датасета: train и test.

In [10]:
train, valid, train_true, valid_true = train_test_split(data,
                                                        targets,
                                                        test_size=0.5, 
                                                        random_state=0)

In [32]:
knn = KNeighborsClassifier(n_neighbors=3)
knn_model = knn.fit(train, train_true)

lr = LogisticRegression(random_state=17)
lr_model = lr.fit(train, train_true)

dtc = DecisionTreeClassifier(max_leaf_nodes=4, random_state=17)
dtc_model = dtc.fit(train, train_true)

svc = SVC(random_state=17)
svc_model = svc.fit(train, train_true)

C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Теперь получим предсказания моделей для второй части тренировочных данных - valid, заполним получившимися метрапризнакми матрицу для обучения мета-алгоритма и обучим его.

In [15]:
models = [knn_model, lr_model, dtc_model, svc_model]
meta_mtrx = np.empty((valid.shape[0], len(models)))  #number of objs, 4 algos

for n, model in enumerate(models):
    meta_mtrx[:, n] = model.predict(valid)
    predicted = model.predict(X_test)
    print(f'{n} auc: {roc_auc_score(y_test, predicted)}')
    
meta = XGBClassifier(n_estimators=40)
meta_model = meta.fit(meta_mtrx, valid_true)

0 auc: 0.7158054453599159
1 auc: 0.7720348896348287
2 auc: 0.7624563455735259
3 auc: 0.6128403349947444


Получим метапризнаки базовых алгоритмов для тестовых данных, чтобы мета-алгоритм мог по ним сделать предсказания.

In [25]:
meta_mtrx_test = np.empty((X_test.shape[0], len(models)))

for n, model in enumerate(models):
    meta_mtrx_test[:, n] = model.predict(X_test)
    
meta_predict = meta.predict(meta_mtrx_test)
print(f'Stacking AUC: {roc_auc_score(y_test, meta_predict)}')

Stacking AUC: 0.772195944800461


Как мы видим, в этом случае финальный скор АUC не превышает рез-т лучших базовых алгов. Попробуем убрать 2 лучших базовых алга, посмотрим, что получится.

In [27]:
models = [knn_model, svc_model]
meta_mtrx = np.empty((valid.shape[0], len(models)))  #number of objs, 2 algos

for n, model in enumerate(models):
    meta_mtrx[:, n] = model.predict(valid)
    predicted = model.predict(X_test)
    print(f'{n} auc: {roc_auc_score(y_test, predicted)}')
    
meta = XGBClassifier(n_estimators=40)
meta_model = meta.fit(meta_mtrx, valid_true)

meta_mtrx_test = np.empty((X_test.shape[0], len(models)))

for n, model in enumerate(models):
    meta_mtrx_test[:, n] = model.predict(X_test)
    
meta_predict = meta.predict(meta_mtrx_test)
print(f'Stacking AUC: {roc_auc_score(y_test, meta_predict)}')

0 auc: 0.7158054453599159
1 auc: 0.6128403349947444
Stacking AUC: 0.7224977113213304


А здесь видно, что стекинг 2-ух не самых лучших алгов дает рез-т лучше, чем каждый из них по отдельности.
Поэтому не всегда построение ансамбля улучшает эффективность алгоритма, все-таки, стекинг - это не панацея.
Тем более, что ансамбль и вычислительнее более сложный, и поддерживается сложнее.